# 유튜브 랭킹 데이터 수집해서 시각화하기

In [1]:
!apt-get update > /dev/null
!pip install selenium > /dev/null
!apt install chromium-chromedriver > /dev/null

In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd

In [47]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')   # 화면없이 실행
options.add_argument('--no-sandbox')
options.add_argument("--single-process")
options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome('chromedriver', options=options)

In [9]:
url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page=1'
driver.get(url)
# time.sleep(2)

In [15]:
trs = driver.find_elements_by_css_selector('.aos-init')
len(trs)

100

In [16]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [23]:
soup.select('form > table > tbody > tr')
channel_list = soup.select('.aos-init')
len(channel_list)

100

In [25]:
channel = channel_list[0]
category = channel.select_one('p.category').get_text().strip()
category

'[음악/댄스/가수]'

In [26]:
name = channel.select_one('.subject a').text.strip()
name

'BLACKPINK'

In [27]:
subscriber = channel.select_one('.subscriber_cnt').text
view = channel.select_one('.view_cnt').text
video = channel.select_one('.video_cnt').text[:-1]
subscriber, view, video

('6400만', '190억0381만', '371')

In [29]:
channels = []

for channel in channel_list:
    category = channel.select_one('p.category').get_text().strip()
    name = channel.select_one('.subject a').text.strip()
    subscriber = channel.select_one('.subscriber_cnt').text
    view = channel.select_one('.view_cnt').text
    video = channel.select_one('.video_cnt').text[:-1]

    channels.append([category, name, subscriber, view, video])

In [30]:
df = pd.DataFrame(channels, columns=['cat','cha','sub','view','vid'])
df

,cat,cha,sub,view,vid
0,[음악/댄스/가수],BLACKPINK,6400만,190억0381만,371
1,[음악/댄스/가수],HYBE LABELS,6030만,187억1305만,654
2,[음악/댄스/가수],BANGTANTV,5640만,121억9496만,"1,579"
3,[음악/댄스/가수],SMTOWN,2850만,218억7092만,"3,729"
4,[키즈/어린이],Boram Tube Vlog [보람튜브 브이로그],2650만,110억5288만,223
...,...,...,...,...,...
95,[음식/요리/레시피],까니짱 [ G-NI ],408만,10억3471만,482
96,[키즈/어린이],로미유 스토리[Romiyu Story],408만,10억6557만,433
97,[음악/댄스/가수],NCT DREAM,386만,3억7422만,228
98,[BJ/인물/연예인],허팝Heopop,381만,31억3892만,"1,847"


- 만과 억을 숫자로

In [34]:
def convert_unit(s):
    #s = ''.join(s.split('억'))
    s = s.replace('억', '').replace('개', '').replace(',', '')
    s = s.replace('만', '0000')
    return f'{int(s):,d}'

convert_unit('2억9,000만개')

'290,000,000'

- 두번째 페이지 - XPath로 찾아 Click() 하여 이동하기

In [36]:
# //*[@id="list-skin"]/nav/span/a[1]
# //*[@id="list-skin"]/nav/span/a[2]
driver.find_element_by_xpath('//*[@id="list-skin"]/nav/span/a[1]').click()

In [38]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [39]:
channel_list = soup.select('.aos-init')
len(channel_list)

100

In [40]:
results = []
for channel in channel_list:
    category = channel.select_one('p.category').get_text().strip(' \n[]')
    name = channel.select_one('.subject a').text.strip()
    subscriber = convert_unit(channel.select_one('.subscriber_cnt').text)
    view = convert_unit(channel.select_one('.view_cnt').text)
    video = convert_unit(channel.select_one('.video_cnt').text)
    results.append([category,name,subscriber,view,video])

In [41]:
df = pd.DataFrame(results, columns=['카테고리','채널명','구독자수','조회수','비디오수'])
df.head()

,카테고리,채널명,구독자수,조회수,비디오수
0,애완/반려동물,크림히어로즈,"3,800,000","1,374,770,000",940
1,BJ/인물/연예인,워크맨-Workman,"3,780,000","679,370,000",141
2,BJ/인물/연예인,waveya 2011,"3,730,000","1,346,280,000",702
3,키즈/어린이,Lime Tube[라임튜브],"3,730,000","2,595,420,000","1,723"
4,음악/댄스/가수,WINNER,"3,730,000","1,171,260,000",428


# page 1에서 10까지 크롤링하기

In [48]:
results = []
for page in range(1,11):
    url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page='+str(page)
    driver.get(url)
    time.sleep(3)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    channel_list = soup.select('.aos-init')

    for channel in channel_list:
        category = channel.select_one('p.category').get_text().strip(' \n[]')
        name = channel.select_one('.subject a').text.strip()
        subscriber = convert_unit(channel.select_one('.subscriber_cnt').text)
        view = convert_unit(channel.select_one('.view_cnt').text)
        video = convert_unit(channel.select_one('.video_cnt').text)
        results.append([category,name,subscriber,view,video])

In [43]:
df = pd.DataFrame(results, columns=['카테고리','채널명','구독자수','조회수','비디오수'])
df.head()

,카테고리,채널명,구독자수,조회수,비디오수
0,음악/댄스/가수,BLACKPINK,"64,000,000","19,003,810,000",371
1,음악/댄스/가수,HYBE LABELS,"60,300,000","18,713,050,000",654
2,음악/댄스/가수,BANGTANTV,"56,400,000","12,194,960,000","1,579"
3,음악/댄스/가수,SMTOWN,"28,500,000","21,870,920,000","3,729"
4,키즈/어린이,Boram Tube Vlog [보람튜브 브이로그],"26,500,000","11,052,880,000",223


In [44]:
df.to_csv('유튜브_순위.csv', index=None)
driver.close()